## Demo data preprocess
**This is a demo for data preprocessing in the utility function provided in baseModule/getStockData.py**

In [1]:
import pandas as pd
import numpy as np

In [2]:
stock_RIC = "AAPL.O"

AGGRE_MIN = 10

raw_data = pd.read_csv("data/NSQ_1min.csv", parse_dates=["Date"])
close = pd.read_csv("data/NSQ_close.csv", parse_dates=["Local_Date_Time"])
target_data = raw_data.loc[raw_data.RIC == stock_RIC]
target_data.head(5)

,BarTime,Date,Time,RIC,Open,High,Low,Close,VWAP,NumberOfTrades,Volume
0,2020-09-01 13:30:00+00:00,2020-09-01,09:30:00,AAPL.O,132.76,132.92,131.49,131.6700,132.656,7032,5143217.0
1,2020-09-01 13:31:00+00:00,2020-09-01,09:31:00,AAPL.O,131.71,132.00,131.32,131.6852,131.696,4657,1089138.0
2,2020-09-01 13:32:00+00:00,2020-09-01,09:32:00,AAPL.O,131.66,132.00,131.13,131.8100,131.554,4668,1062337.0
3,2020-09-01 13:33:00+00:00,2020-09-01,09:33:00,AAPL.O,131.81,131.83,130.81,130.9047,131.158,5074,1147498.0
4,2020-09-01 13:34:00+00:00,2020-09-01,09:34:00,AAPL.O,130.91,131.39,130.89,131.3300,131.134,4179,915152.0


In [3]:
data = pd.merge(target_data, close,
                left_on=["Date", "RIC"], right_on=["Local_Date_Time", "RIC"],
                how="inner", suffixes=(None, "_atClose"))
data["TimeIndex"] = data["Date"].astype(str) + " " + data["Time"]
data["TimeIndex"] = pd.to_datetime(data["TimeIndex"])
data = data.set_index("TimeIndex")
data.head(5)

,BarTime,Date,Time,RIC,Open,High,Low,Close,VWAP,NumberOfTrades,Volume,Local_Date_Time,Price,Volume_atClose
TimeIndex,,,,,,,,,,,,,,
2020-09-01 09:30:00,2020-09-01 13:30:00+00:00,2020-09-01,09:30:00,AAPL.O,132.76,132.92,131.49,131.6700,132.656,7032,5143217.0,2020-09-01,134.18,6041734
2020-09-01 09:31:00,2020-09-01 13:31:00+00:00,2020-09-01,09:31:00,AAPL.O,131.71,132.00,131.32,131.6852,131.696,4657,1089138.0,2020-09-01,134.18,6041734
2020-09-01 09:32:00,2020-09-01 13:32:00+00:00,2020-09-01,09:32:00,AAPL.O,131.66,132.00,131.13,131.8100,131.554,4668,1062337.0,2020-09-01,134.18,6041734
2020-09-01 09:33:00,2020-09-01 13:33:00+00:00,2020-09-01,09:33:00,AAPL.O,131.81,131.83,130.81,130.9047,131.158,5074,1147498.0,2020-09-01,134.18,6041734
2020-09-01 09:34:00,2020-09-01 13:34:00+00:00,2020-09-01,09:34:00,AAPL.O,130.91,131.39,130.89,131.3300,131.134,4179,915152.0,2020-09-01,134.18,6041734


In [4]:
## Resampling with given number of minutes aggregations (AGGRE MIN)

resample_data = pd.DataFrame()
resample_data["High"] = data["High"].resample("{}T".format(AGGRE_MIN), origin="start_day").max()
resample_data["Low"] = data["Low"].resample("{}T".format(AGGRE_MIN), origin="start_day").min()
resample_data["Open"] = data["Open"].resample("{}T".format(AGGRE_MIN), origin="start_day").first()
resample_data["Close"] = data["Close"].resample("{}T".format(AGGRE_MIN), origin="start_day").last()
resample_data["Auction"] = data["Price"].resample("{}T".format(AGGRE_MIN), origin="start_day").last()
resample_data = resample_data.dropna(how="all")

resample_data["Volume"] = data["Volume"].resample("{}T".format(AGGRE_MIN), origin="start_day").sum()
resample_data["Date"] = pd.to_datetime(resample_data.index.date)
resample_data["RIC"] = stock_RIC

## Dropping days with abnormal trading minutes (# min != 390)
day_check = target_data.groupby("Date")["Time"].count()
abnormal_day = day_check[day_check != 390].index.values
resample_data = resample_data.loc[~resample_data.Date.isin(abnormal_day)]

resample_data["c/o_Vol"] = abs(np.log(resample_data["Close"]/resample_data["Open"]))
resample_data["h/l_Vol"] = np.log(resample_data["High"]/resample_data["Low"])

resample_data.reset_index(inplace=True, drop=False)
## Shifting TimeIndex so hh:mm:ss represents the 10 min intervals prior of that.
## For example, TimeIndex 09:40:00 represents the time range 09:30:00 to 09:39:00
resample_data["TimeIndex"] = resample_data["TimeIndex"] + pd.to_timedelta("{} minutes".format(AGGRE_MIN))

## Calculate daily average volatility measure
sqrt_mse = lambda x: np.sqrt((x ** 2).sum())
co_coef = np.sqrt(resample_data.groupby("Date")["c/o_Vol"].count())
hl_coef = np.sqrt(resample_data.groupby("Date")["h/l_Vol"].count())
day_coVol = (resample_data.groupby("Date")["c/o_Vol"].apply(sqrt_mse) * co_coef).reset_index(drop=False)
day_hlVol = (resample_data.groupby("Date")["h/l_Vol"].apply(sqrt_mse) * hl_coef).reset_index(drop=False)

resample_data = pd.merge(resample_data, day_coVol, left_on="Date", right_on="Date", how="inner", suffixes=(None, "_Day"))
resample_data = pd.merge(resample_data, day_hlVol, left_on="Date", right_on="Date", how="inner", suffixes=(None, "_Day"))

In [5]:
data_350 = data.loc[(data.index.hour==15) & (data.index.minute==50) & (data.index.second==0)]
full_data = pd.merge(resample_data, data_350[["Date", "RIC", "Open"]],
             left_on=["Date", "RIC"],
             right_on=["Date", "RIC"],
             how="inner", suffixes=(None, "_at_3:50"))

full_data["Auction_logdiff"] = abs(np.log(full_data["Auction"] / full_data["Open_at_3:50"]))
display(full_data)

,TimeIndex,High,Low,Open,Close,Auction,Volume,Date,RIC,c/o_Vol,h/l_Vol,c/o_Vol_Day,h/l_Vol_Day,Open_at_3:50,Auction_logdiff
0,2020-09-01 09:40:00,132.920,130.5300,132.7600,130.7350,134.18,13108069.0,2020-09-01,AAPL.O,0.015371,0.018144,0.129724,0.206046,133.79,0.002911
1,2020-09-01 09:50:00,131.850,130.7001,130.7500,131.1177,134.18,6229895.0,2020-09-01,AAPL.O,0.002808,0.008760,0.129724,0.206046,133.79,0.002911
2,2020-09-01 10:00:00,132.040,130.9700,131.1116,131.6133,134.18,4661043.0,2020-09-01,AAPL.O,0.003819,0.008137,0.129724,0.206046,133.79,0.002911
3,2020-09-01 10:10:00,132.080,131.3100,131.6400,131.3700,134.18,4047447.0,2020-09-01,AAPL.O,0.002053,0.005847,0.129724,0.206046,133.79,0.002911
4,2020-09-01 10:20:00,131.480,130.7700,131.3900,131.3750,134.18,4030219.0,2020-09-01,AAPL.O,0.000114,0.005415,0.129724,0.206046,133.79,0.002911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9784,2021-09-01 15:20:00,153.160,152.7900,152.8800,153.1487,152.51,1178189.0,2021-09-01,AAPL.O,0.001756,0.002419,0.079192,0.126208,152.59,0.000524
9785,2021-09-01 15:30:00,153.175,152.9519,153.1400,152.9550,152.51,974606.0,2021-09-01,AAPL.O,0.001209,0.001458,0.079192,0.126208,152.59,0.000524
9786,2021-09-01 15:40:00,152.970,152.4550,152.9550,152.4800,152.51,2099024.0,2021-09-01,AAPL.O,0.003110,0.003372,0.079192,0.126208,152.59,0.000524
9787,2021-09-01 15:50:00,152.660,152.4600,152.4800,152.5850,152.51,1679427.0,2021-09-01,AAPL.O,0.000688,0.001311,0.079192,0.126208,152.59,0.000524


In [6]:
full_data.isna().sum()

TimeIndex          0
High               0
Low                0
Open               0
Close              0
Auction            0
Volume             0
Date               0
RIC                0
c/o_Vol            0
h/l_Vol            0
c/o_Vol_Day        0
h/l_Vol_Day        0
Open_at_3:50       0
Auction_logdiff    0
dtype: int64